In [7]:
import numpy as np 
import pandas as pd
import torch

DBpedia

In [4]:
data_dir ="../all_data/dbpedia_csv/train.csv"

df = pd.read_csv(f"{data_dir}", header= None)

In [5]:
df.head()

,0,1,2
0,1,E. D. Abbott Ltd,Abbott of Farnham E D Abbott Limited was a Br...
1,1,Schwan-Stabilo,Schwan-STABILO is a German maker of pens for ...
2,1,Q-workshop,Q-workshop is a Polish company located in Poz...
3,1,Marvell Software Solutions Israel,Marvell Software Solutions Israel known as RA...
4,1,Bergan Mercy Medical Center,Bergan Mercy Medical Center is a hospital loc...


In [16]:
df[0].value_counts()

1     40000
2     40000
3     40000
4     40000
5     40000
6     40000
7     40000
8     40000
9     40000
10    40000
11    40000
12    40000
13    40000
14    40000
Name: 0, dtype: int64

In [12]:
df.shape

(560000, 3)

In [8]:
# load in the preprocessed tar file

# load data
preprocessed_data = torch.load("../all_data/dbpedia_csv/processed/TRAIN_data.pth.tar")

In [14]:
len(preprocessed_data['sents'])

560000

In [21]:
preprocessed_data['labels'][5]

0